# Задание 1

In [60]:
import urllib.request as urllib2
from bs4 import *
from urllib.parse  import urljoin

urlList = set()


def crawl(pages, depth=None):
    indexed_url = [] # a list for the main and sub-HTML websites in the main website
    for i in range(depth):
        for page in pages:
            if page not in indexed_url:
                indexed_url.append(page)
                try:
                    c = urllib2.urlopen(page)
                except:
                    print( "Could not open %s" % page)
                    continue
                soup = BeautifulSoup(c.read())
                links = soup('a') #finding all the sub_links
                for link in links:
                    if 'href' in dict(link.attrs):
                        url = urljoin(page, link['href'])
                        if url.find("'") != -1:
                                continue
                        url = url.split('#')[0] 
                        if url[0:4] == 'http':
                                indexed_url.append(url)
        pages = indexed_url
    return indexed_url

while len(urlList) < 100:
    pagelist=["https://en.wikipedia.org/wiki/Python_%28programming_language%29"]
    urls = crawl(pagelist, depth=1)
    urlList.update(urls)
    
result = []
if len(urlList) > 500:
    x = 0
    for e in urlList:
        result.append(e)
        x+=1
        if(x == 500):
            break
# некоторые ссылки могут быть битые, поэтому заранне сохраним 500 страниц
with open('search/links.txt', 'w') as f:
    for item in result:
        f.write("%s\n" % item)

In [85]:
import urllib
from random import randrange
counter = 0
with open('search/index.txt', 'w') as f:
    with open('search/links.txt') as file:
        for line in file:
            name = 'search/task1/' + str(randrange(5000)) + '.html'
            try:
                urllib.request.urlretrieve(line, name)
                counter += 1
                print(counter)
                result_string = line.replace("\n", "") + ',' + name + '\n'
                f.write(result_string)
                if(counter == 100):
                    print('Time to break up')
                    f.close()
                    break
            except:
                print( "Could not open %s" % line)
                continue

1
2
3
4
5
6
7
8
9
10
11
12
Could not open http://ebeab.com/2014/01/21/python-culture/

13
14
15
Could not open http://effbot.org/zone/call-by-object.htm

16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
Could not open http://www.immunitysec.com/products-immdbg.shtml

77
78
79
Could not open http://gimp-win.sourceforge.net/faq.html

80
81
Could not open https://web.archive.org/web/20210813194305/https://w3techs.com/technologies/details/pl-python

82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
Time to break up


# Задание 2

In [62]:
from bs4 import BeautifulSoup
import nltk
import re
nltk.download('punkt')
nltk.download("stopwords")

from nltk import sent_tokenize
from nltk.corpus import stopwords

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

#получаем список стоп слов
stop = stopwords.words('english')

CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/simononboard/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/simononboard/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/simononboard/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [87]:
import os
arr = os.listdir('search/task1/')
counter = 0
for file in arr:
    with open('search/task1/' + file, 'r', encoding="ISO-8859-1") as fp:
        tokens = []
        dictionary = {}
        data = fp.read()
        data = re.sub("<head>.*?</head>", "", data, flags=re.DOTALL)
        data = re.sub("<nav.*>.*?</nav.*>", "", data, flags=re.DOTALL)
        data = re.sub("<footer>.*?</footer>", "", data, flags=re.DOTALL)
        cleantext = re.sub(CLEANR, ' ', data)
        lines = [cleantext]
        if(cleantext.find('\n') != -1):
            lines = cleantext.split('\n')
            for line in lines:
                if line:
                    sentences = sent_tokenize(line, language="english")
                    for cur in sentences:
                        cur = cur.lower()
                        cur = re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", cur)
                        cur = re.sub(r"\d+", " ", cur)
                        cur = '  '.join([word for word in cur.split() if word not in (stop)])
                        words = cur.split()
                        for word in words:
                            current_lemma = WordNetLemmatizer().lemmatize(word)
                            tokens.append(word)
                            if current_lemma not in dictionary:
                                dictionary[current_lemma] = set()
                            dictionary[current_lemma].add(word)   
        with open('search/task2/' + file.replace(".html", "") + "_tokens" + '.txt', 'w') as f:
            for item in tokens:
                f.write("%s\n" % item)
        with open('search/task2/' + file.replace(".html", "") + "_dictionary" + '.txt', 'w') as f:
            for key, value in dictionary.items():
                result_string = key + ": "
                if( len(value) >= 2 ) :
                    print( key + ": " + str(value))
                for val in value:
                    result_string += val + " "
                f.write("%s\n" % result_string)    
        counter += 1

print(counter)

element: {'element', 'elements'}
sequence: {'sequence', 'sequences'}
index: {'indices', 'index'}
number: {'number', 'numbers'}
case: {'case', 'cases'}
object: {'object', 'objects'}
value: {'values', 'value'}
term: {'terms', 'term'}
context: {'contexts', 'context'}
common: {'common', 'commons'}
language: {'language', 'languages'}
computer: {'computers', 'computer'}
array: {'arrays', 'array'}
start: {'starts', 'start'}
situation: {'situation', 'situations'}
instance: {'instances', 'instance'}
time: {'time', 'times'}
second: {'second', 'seconds'}
content: {'content', 'contents'}
origin: {'origin', 'origins'}
property: {'properties', 'property'}
reference: {'reference', 'references'}
c: {'cs', 'c'}
natural: {'naturals', 'natural'}
address: {'addresses', 'address'}
access: {'access', 'accesses'}
run: {'run', 'runs'}
w: {'w', 'ws'}
design: {'design', 'designs'}
inequality: {'inequality', 'inequalities'}
range: {'range', 'ranges'}
interval: {'interval', 'intervals'}
convention: {'convention',

page: {'pages', 'page'}
capture: {'capture', 'captures'}
system: {'system', 'systems'}
article: {'articles', 'article'}
reference: {'reference', 'references'}
source: {'source', 'sources'}
support: {'support', 'supports'}
user: {'users', 'user'}
workflow: {'workflows', 'workflow'}
feature: {'features', 'feature'}
link: {'link', 'links'}
program: {'program', 'programs'}
run: {'run', 'runs'}
line: {'lines', 'line'}
block: {'block', 'blocks'}
server: {'servers', 'server'}
list: {'list', 'lists'}
box: {'box', 'boxes'}
report: {'report', 'reports'}
column: {'columns', 'column'}
c: {'cs', 'c'}
module: {'modules', 'module'}
comparison: {'comparisons', 'comparison'}
environment: {'environments', 'environment'}
package: {'package', 'packages'}
style: {'styles', 'style'}
link: {'link', 'links'}
article: {'articles', 'article'}
table: {'table', 'tables'}
tool: {'tool', 'tools'}
editor: {'editors', 'editor'}
builder: {'builder', 'builders'}
language: {'language', 'languages'}
content: {'content', 

string: {'string', 'strings'}
literal: {'literals', 'literal'}
notation: {'notations', 'notation'}
type: {'types', 'type'}
value: {'values', 'value'}
language: {'language', 'languages'}
sequence: {'sequence', 'sequences'}
character: {'characters', 'character'}
quote: {'quotes', 'quote'}
part: {'parts', 'part'}
escape: {'escape', 'escapes'}
problem: {'problems', 'problem'}
case: {'case', 'cases'}
content: {'content', 'contents'}
function: {'function', 'functions'}
concatenation: {'concatenation', 'concatenations'}
kind: {'kind', 'kinds'}
variable: {'variables', 'variable'}
note: {'note', 'notes'}
reference: {'reference', 'references'}
link: {'link', 'links'}
bracket: {'brackets', 'bracket'}
quotation: {'quotation', 'quotations'}
common: {'common', 'commons'}
begin: {'begins', 'begin'}
mark: {'marks', 'mark'}
term: {'terms', 'term'}
expression: {'expressions', 'expression'}
number: {'number', 'numbers'}
example: {'examples', 'example'}
apostrophe: {'apostrophe', 'apostrophes'}
brace: {'b

system: {'system', 'systems'}
col: {'col', 'cols'}
image: {'images', 'image'}
utility: {'utilities', 'utility'}
developer: {'developers', 'developer'}
c: {'cs', 'c'}
state: {'state', 'states'}
source: {'source', 'sources'}
release: {'releases', 'release'}
year: {'year', 'years'}
repository: {'repository', 'repositories'}
package: {'package', 'packages'}
manager: {'managers', 'manager'}
tool: {'tool', 'tools'}
platform: {'platforms', 'platform'}
kernel: {'kernel', 'kernels'}
user: {'users', 'user'}
license: {'license', 'licenses'}
project: {'project', 'projects'}
version: {'versions', 'version'}
subsystem: {'subsystem', 'subsystems'}
firewall: {'firewall', 'firewalls'}
library: {'library', 'libraries'}
window: {'windows', 'window'}
word: {'word', 'words'}
mean: {'mean', 'means'}
support: {'support', 'supports'}
content: {'content', 'contents'}
network: {'networks', 'network'}
computer: {'computers', 'computer'}
server: {'server', 'servers'}
backdoor: {'backdoors', 'backdoor'}
foundation

archive: {'archives', 'archive'}
capture: {'capture', 'captures'}
crawl: {'crawls', 'crawl'}
link: {'link', 'links'}
come: {'come', 'comes'}
answer: {'answer', 'answers'}
reference: {'reference', 'references'}
one: {'ones', 'one'}
statement: {'statement', 'statements'}
work: {'work', 'works'}
value: {'values', 'value'}
dictionary: {'dictionaries', 'dictionary'}
change: {'changes', 'change'}
list: {'list', 'lists'}
get: {'gets', 'get'}
look: {'look', 'looks'}
case: {'case', 'cases'}
copy: {'copy', 'copies'}
type: {'types', 'type'}
say: {'say', 'says'}
poster: {'poster', 'posters'}
misconception: {'misconceptions', 'misconception'}
way: {'way', 'ways'}
name: {'names', 'name'}
object: {'object', 'objects'}
end: {'ends', 'end'}
drawing: {'drawings', 'drawing'}
variable: {'variables', 'variable'}
matter: {'matter', 'matters'}
picture: {'pictures', 'picture'}
surprise: {'surprises', 'surprise'}
point: {'points', 'point'}
return: {'return', 'returns'}
situation: {'situation', 'situations'}
sl

...

index: {'index', 'indexes'}
module: {'modules', 'module'}
author: {'author', 'authors'}
release: {'releases', 'release'}
date: {'dates', 'date'}
feature: {'features', 'feature'}
pep: {'peps', 'pep'}
path: {'path', 'paths'}
language: {'language', 'languages'}
code: {'codes', 'code'}
try: {'tries', 'try'}
function: {'function', 'functions'}
switch: {'switch', 'switches'}
usage: {'usage', 'usages'}
package: {'package', 'packages'}
way: {'way', 'ways'}
see: {'sees', 'see'}
number: {'number', 'numbers'}
source: {'source', 'sources'}
search: {'search', 'searches'}
change: {'changes', 'change'}
log: {'logs', 'log'}
find: {'finds', 'find'}
patch: {'patches', 'patch'}
bug: {'bug', 'bugs'}
figure: {'figure', 'figures'}
attempt: {'attempts', 'attempt'}
detail: {'detail', 'details'}
implementation: {'implementations', 'implementation'}
item: {'item', 'items'}
version: {'versions', 'version'}
break: {'break', 'breaks'}
need: {'need', 'needs'}
conversion: {'conversions', 'conversion'}
order: {'order

n: {'ns', 'n'}
application: {'applications', 'application'}
article: {'articles', 'article'}
citation: {'citations', 'citation'}
source: {'source', 'sources'}
macintosh: {'macintosh', 'macintoshes'}
user: {'users', 'user'}
column: {'columns', 'column'}
integer: {'integer', 'integers'}
number: {'number', 'numbers'}
string: {'string', 'strings'}
category: {'categories', 'category'}
case: {'case', 'cases'}
anova: {'anova', 'anovas'}
value: {'values', 'value'}
menu: {'menu', 'menus'}
pc: {'pcs', 'pc'}
window: {'windows', 'window'}
content: {'content', 'contents'}
run: {'run', 'runs'}
measure: {'measures', 'measure'}
factor: {'factor', 'factors'}
system: {'system', 'systems'}
format: {'format', 'formats'}
style: {'styles', 'style'}
c: {'cs', 'c'}
review: {'review', 'reviews'}
contract: {'contracts', 'contract'}
free: {'free', 'frees'}
approach: {'approach', 'approaches'}
designer: {'designers', 'designer'}
specification: {'specifications', 'specification'}
component: {'component', 'componen

website: {'website', 'websites'}
award: {'award', 'awards'}
change: {'changes', 'change'}
product: {'products', 'product'}
framework: {'framework', 'frameworks'}
page: {'page', 'pages'}
year: {'year', 'years'}
language: {'language', 'languages'}
ranking: {'rankings', 'ranking'}
engine: {'engine', 'engines'}
month: {'months', 'month'}
difference: {'difference', 'differences'}
course: {'courses', 'course'}
check: {'checks', 'check'}
system: {'system', 'systems'}
position: {'position', 'positions'}
lot: {'lot', 'lots'}
mail: {'mails', 'mail'}
term: {'terms', 'term'}
winner: {'winner', 'winners'}
query: {'query', 'queries'}
measure: {'measure', 'measures'}
intersection: {'intersections', 'intersection'}
come: {'come', 'comes'}
id: {'ids', 'id'}
generator: {'generators', 'generator'}
style: {'styles', 'style'}
article: {'articles', 'article'}
need: {'need', 'needs'}
citation: {'citations', 'citation'}
help: {'help', 'helps'}
source: {'source', 'sources'}
book: {'books', 'book'}
routine: {'r

pep: {'peps', 'pep'}
capture: {'capture', 'captures'}
transformer: {'transformer', 'transformers'}
machine: {'machine', 'machines'}
model: {'model', 'models'}
language: {'language', 'languages'}
output: {'outputs', 'output'}
weight: {'weight', 'weights'}
content: {'content', 'contents'}
list: {'list', 'lists'}
c: {'cs', 'c'}
part: {'parts', 'part'}
problem: {'problems', 'problem'}
network: {'networks', 'network'}
vector: {'vectors', 'vector'}
mean: {'mean', 'means'}
field: {'field', 'fields'}
word: {'word', 'words'}
block: {'blocks', 'block'}
mechanism: {'mechanisms', 'mechanism'}
input: {'input', 'inputs'}
task: {'task', 'tasks'}
process: {'process', 'processes'}
sequence: {'sequence', 'sequences'}
time: {'time', 'times'}
system: {'system', 'systems'}
representation: {'representation', 'representations'}
common: {'common', 'commons'}
architecture: {'architectures', 'architecture'}
decoder: {'decoder', 'decoders'}
head: {'head', 'heads'}
alternative: {'alternatives', 'alternative'}
app

capture: {'capture', 'captures'}
template: {'templates', 'template'}
language: {'language', 'languages'}
content: {'content', 'contents'}
editor: {'editors', 'editor'}
compromise: {'compromise', 'compromises'}
criterion: {'criteria', 'criterion'}
list: {'list', 'lists'}
style: {'styles', 'style'}
bracket: {'brackets', 'bracket'}
generation: {'generation', 'generations'}
type: {'types', 'type'}
second: {'second', 'seconds'}
approach: {'approach', 'approaches'}
system: {'system', 'systems'}
compiler: {'compiler', 'compilers'}
infrastructure: {'infrastructures', 'infrastructure'}
fundamental: {'fundamentals', 'fundamental'}
product: {'products', 'product'}
technology: {'technology', 'technologies'}
language: {'language', 'languages'}
team: {'team', 'teams'}
project: {'project', 'projects'}
experience: {'experiences', 'experience'}
build: {'builds', 'build'}
effort: {'efforts', 'effort'}
deployment: {'deployments', 'deployment'}
paper: {'papers', 'paper'}
sort: {'sort', 'sorts'}
common: {'

# Задание 3

In [67]:
import glob

In [88]:
arr = glob.glob("search/task2/*_tokens.txt")
counter = 0
token_set = set()
tokens_dictionary = {}

result_dictionary = {}

for file in arr:
    with open(file, 'r') as fp:
        data = fp.read()
        name = file.replace("_tokens.txt", "").replace("search/task2/", "")
        data_list = data.splitlines()
        tokens_dictionary[name] = data_list
        token_set.update(data_list)


for token in token_set:
    result_dictionary[token] = []
    for key, value in tokens_dictionary.items():
        if token in value:
            result_dictionary[token].append(key)

            
with open('search/task3/inverted_index.txt', 'w') as f:
            for key, value in result_dictionary.items():
                result_string = key + " "
                for val in value:
                    result_string += val + " "
                f.write("%s\n" % result_string) 


with open('search/task3/inverted_index_2.txt', 'w') as f:
            for key, value in result_dictionary.items():
                result_string = {"count": len(value), "inverted_array": value, "word": key}
                f.write("%s\n" % str(result_string)) 

# Задание 4

In [119]:
import collections
import json
import math

In [124]:
arr = glob.glob("search/task2/*_tokens.txt")
token_corpus_count = {}

tokens_dictionary = {}

def compute_tf(text):
    tf_text = collections.Counter(text)
    for i in tf_text:
        tf_text[i] = tf_text[i]/float(len(text))
    return tf_text

def compute_idf(word, corpus):
    return math.log10(len(tokens_dictionary)/token_corpus_count[word])


with open('search/task3/inverted_index_2.txt', 'r') as f:
    data = f.readlines()
    for elem in data:
        elem = elem.replace("\'", "\"")
        string_data = json.loads(elem)
        token_corpus_count[string_data['word']] = string_data['count']
        
for file in arr:
    with open(file, 'r') as fp:
        data = fp.read()
        data_list = data.splitlines()
        name = file.replace("_tokens.txt", "").replace("search/task2/", "")
        tokens_dictionary[name] = data_list

print(len(tokens_dictionary))

for key, value in tokens_dictionary.items():
    tf_idf_dictionary = {}
    computed_tf = compute_tf(value)
    for word in computed_tf:
        idf = compute_idf(word,token_corpus_count)
        tf_idf_dictionary[word] = [idf, computed_tf[word] * idf]
    with open('search/task4/' + key + '.txt', 'w') as f:
            for key1, value1 in tf_idf_dictionary.items():
                result_string = key1 + " " + str(value1[0]) + " " + str(value1[1])
                f.write("%s\n" % result_string)    


98


In [134]:
arr = glob.glob("search/task2/*_tokens.txt")
token_corpus_count = {}

tokens_dictionary = {}

def compute_tf(text):
    tf_text = collections.Counter(text)
    for i in tf_text:
        tf_text[i] = tf_text[i]/float(len(text))
    return tf_text

def compute_idf(word, corpus):
    return math.log10(len(tokens_dictionary)/token_corpus_count[word])


with open('search/task3/inverted_index_2.txt', 'r') as f:
    data = f.readlines()
    for elem in data:
        elem = elem.replace("\'", "\"")
        string_data = json.loads(elem)
        token_corpus_count[string_data['word']] = string_data['count']
        
for file in arr:
    with open(file, 'r') as fp:
        data = fp.read()
        data_list = data.splitlines()
        name = file.replace("_tokens.txt", "").replace("search/task2/", "")
        tokens_dictionary[name] = data_list

lemma_dictionary = {}
arr = glob.glob("search/task2/*_dictionary.txt")

for file in arr:
    with open(file, 'r') as fp:
        data = fp.read()
        data_list = data.splitlines()
        name = file.replace("_dictionary.txt", "").replace("search/task2/", "")
        lemma_dictionary[name] = data_list
        
for key, value in tokens_dictionary.items():
    tf_idf_dictionary = {}
    computed_tf = compute_tf(value)
    curr_lemmas = lemma_dictionary[key]
    for lemma in curr_lemmas:
        values = lemma.split(' ')
        tf_lemma_sum = float(0)
        idf_sum = float(0)
        for i in range(1,len(values) - 1):
            tf_lemma_sum += computed_tf[values[i]]
            idf_sum += compute_idf(values[i],token_corpus_count)
        tf_idf_dictionary[values[0].replace(":","")] = [tf_lemma_sum, idf_sum, tf_lemma_sum * idf_sum]
    with open('search/task4_1/' + key + '_lemma.txt', 'w') as f:
            for key1, value1 in tf_idf_dictionary.items():
                result_string = key1 + " " + str(value1[0]) + " " + str(value1[1]) + " " str(value1[2])
                f.write("%s\n" % result_string) 